In [1]:
import os
import pandas as pd
import numpy as np
import pickle

import yaml

cfg = None
    #' load config.yaml file in the root dir 
with open("../config.yaml", 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    
# Use 3 decimal places in output display
pd.set_option("display.precision", 3)

# Don't wrap repr(DataFrame) across additional lines
pd.set_option("display.expand_frame_repr", False)

# Set max rows displayed in output to 25
pd.set_option("display.max_rows", 25)

pd.set_option('display.float_format', lambda x: '%.2f' % x)


#Function to get data frame summary
def get_data_frame_summmary(data_frame, sort_by='percent_missing', ascending=False):
    unique_values = data_frame.apply(lambda x: [x.unique()])
    unique_counts = data_frame.apply(lambda x: len(x.unique()))
    percent_missing = data_frame.apply(lambda x: sum(pd.isnull(x))/len(x)*100)
    data_type = data_frame.dtypes

    return pd.DataFrame(dict(unique_values = unique_values, 
                                unique_counts = unique_counts,
                                data_type = data_type,
                                percent_missing = percent_missing,
                                )).reset_index().sort_values(by=sort_by, ascending=ascending)

# Function to drop outliers of numeric columns
def drop_outliers(data_frame, exclude=[]):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    
    for colname in num_cols:
        upper_lim = data_frame[colname].quantile(.95)
        lower_lim = data_frame[colname].quantile(.05)
        
        print(f'Dropping outliers for {colname} upper limit = {upper_lim} and lower limit = {lower_lim}')
        data_frame = data_frame[(data_frame[colname] < upper_lim) & (data_frame[colname] > lower_lim)]
      
    return data_frame


# Function to drop outliers of numeric columns
def scale_numeric_features(data_frame, exclude=[], 
                           method='standardize',
                           inplace=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    num_cols = data_frame.select_dtypes(include=numerics).columns

    num_cols = num_cols.difference(exclude)
    print(f'********************* - Scaling following {len(num_cols)} features - **********************')
    for colname in num_cols:
        new_colname = colname if inplace else colname+'_'+method+'d'
        print(f' {colname} {method}d to {new_colname}')
        
        if method == 'standardize':              
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].mean()) / data_frame[colname].std()
        elif method == 'normalize':
              data_frame[new_colname] = (data_frame[colname] - data_frame[colname].min()) / (data_frame[colname].max() - data_frame[colname].min())
        else:
              print(f'Unknown method {method} specified, please select one of "standardize" or "normalize"')

      
    return data_frame


def remove_items(iteamlist, removelist):
    return [ele for ele in iteamlist if ele not in removelist]

def diff(first, second):
    second = set(second)
    return [item for item in first if item not in second]
    
def months_between_columns(date_series1, date_series2):
    return round((date_series1-date_series2) / np.timedelta64(1, 'M'))
    


In [2]:
# Read Data
interaction = pd.read_pickle('../data/pickle/interaction.pickle')

df = interaction
del interaction

df.head()

,CHANNEL,CONTACT_COUNTRY,CONTACT_REASON_LVL1_DESC,CONTACT_REASON_LVL2_DESC,CONTACT_REASON_LVL3_DESC,CONTACT_TYPE,CREATED_TO_CLOSED_DAYS,CREATED_TO_INITIAL_RESPONSE_DAYS,CUSTOMER_CLASSIFICATION_PRODUCT,CUSTOMER_CLASSIFICATION_ROLE,...,INCIDENT_SYSTEM,NUMBER_OF_RESPONSES,OWNER_ID,OWNER_NAME,RESOLUTION_CODE_LVL1_DESC,RESOLUTION_CODE_LVL2_DESC,RESOLUTION_CODE_LVL3_DESC,SOURCE_LVL1_DESC,SOURCE_LVL2_DESC,STATUS
0,Phone,No Value,Using the product,Error message,No Value,No Value,0.00,nan,HESI - iNet Testing,Organization,...,No Value,0,HOCS Education REPH 1LS,Switzel Marie Jaena,Provided product information/doc,No Value,No Value,CX Console,Incident Editor,Solved
1,Phone,United States,Access,Setup / manage,No Value,No Value,0.00,nan,HESI Legacy,Organization,...,No Value,0,HOCS Education REPH 1LS,Switzel Marie Jaena,Provided product information/doc,No Value,No Value,CX Console,Incident Editor,Solved
2,CSS Email,United States,Review,Additional info required,No Value,No Value,0.83,0.83,Journal,Researcher,...,Evise,1,GJP Journal Managers Chennai,Latha Sudarsanam,Didn't use KB article,No Value,No Value,Utilities,Techmail - Service Mailbox,Solved
3,CSS Email,No Value,No Contact Reason,No Contact Reason,No Contact Reason,No Value,0.59,nan,No Product,Content,...,No Value,0,No Agent,No Agent,No Resolution Code,No Resolution Code,No Resolution Code,Utilities,Techmail - Service Mailbox,Solved
4,No Value,No Value,Author Profile Correction,Reprofile,No Value,No Value,1.37,nan,Scopus,Content,...,No Value,0,No Agent,No Agent,AFW tool,No Value,No Value,Public API,Connect Web Services - SOAP,Solved


# Replace strings indicating missing data with null

In [3]:
#replace strings indicating missing data with null
df.replace(['nan', 'N.A', 'N.A.', 'NaN', 'Nan', '00-00-00', '0-00-00', 'Unknown', 'No Value', 'No Agent', 'No Contact Reason', 'No Customer Classification', 'No Resolution Code'], np.nan, inplace=True)
df

,CHANNEL,CONTACT_COUNTRY,CONTACT_REASON_LVL1_DESC,CONTACT_REASON_LVL2_DESC,CONTACT_REASON_LVL3_DESC,CONTACT_TYPE,CREATED_TO_CLOSED_DAYS,CREATED_TO_INITIAL_RESPONSE_DAYS,CUSTOMER_CLASSIFICATION_PRODUCT,CUSTOMER_CLASSIFICATION_ROLE,...,INCIDENT_SYSTEM,NUMBER_OF_RESPONSES,OWNER_ID,OWNER_NAME,RESOLUTION_CODE_LVL1_DESC,RESOLUTION_CODE_LVL2_DESC,RESOLUTION_CODE_LVL3_DESC,SOURCE_LVL1_DESC,SOURCE_LVL2_DESC,STATUS
0,Phone,NaN,Using the product,Error message,NaN,NaN,0.00,nan,HESI - iNet Testing,Organization,...,NaN,0,HOCS Education REPH 1LS,Switzel Marie Jaena,Provided product information/doc,NaN,NaN,CX Console,Incident Editor,Solved
1,Phone,United States,Access,Setup / manage,NaN,NaN,0.00,nan,HESI Legacy,Organization,...,NaN,0,HOCS Education REPH 1LS,Switzel Marie Jaena,Provided product information/doc,NaN,NaN,CX Console,Incident Editor,Solved
2,CSS Email,United States,Review,Additional info required,NaN,NaN,0.83,0.83,Journal,Researcher,...,Evise,1,GJP Journal Managers Chennai,Latha Sudarsanam,Didn't use KB article,NaN,NaN,Utilities,Techmail - Service Mailbox,Solved
3,CSS Email,NaN,NaN,NaN,NaN,NaN,0.59,nan,No Product,Content,...,NaN,0,NaN,NaN,NaN,NaN,NaN,Utilities,Techmail - Service Mailbox,Solved
4,NaN,NaN,Author Profile Correction,Reprofile,NaN,NaN,1.37,nan,Scopus,Content,...,NaN,0,NaN,NaN,AFW tool,NaN,NaN,Public API,Connect Web Services - SOAP,Solved
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
575001,Web,NaN,Author Profile Correction,Reprofile,NaN,NaN,6.52,nan,Scopus,Content,...,NaN,0,AFT,Lakshmi Lingaraju,AFW tool,NaN,NaN,Public API,Connect Web Services - SOAP,Solved
575002,CSS Email,NaN,Access,Cannot login,NaN,Employee of Academic Institute (B2B),2.38,nan,Evolve - Portal/Catalog,Organization,...,NaN,0,HOCS Clinical Solutions STL CWR,Trevon Allen,Resolution Code Not Found,NaN,NaN,Utilities,Techmail - Service Mailbox,Solved
575003,Web,NaN,Author Profile Correction,Reprofile,NaN,NaN,11.72,nan,Scopus,Content,...,NaN,0,AFT,Nanda Thararaj H. B,AFW tool,NaN,NaN,Public API,Connect Web Services - SOAP,Solved
575004,CSS Email,United States,Review,Check paper status,NaN,Individual (B2C),6.67,nan,Journal,Researcher,...,EES,0,GJP Journal Managers Chennai,Jaya Pavithra Bhaskaran,Sent to Editor to resolve,NaN,NaN,Utilities,Techmail - Service Mailbox,Solved


In [4]:
for i in range(0,df.shape[1]):
    print(df.iloc[:,i].value_counts())

CSS Email       313620
Web             145082
Phone            52337
Chat             31850
Email             5056
Callback          1133
Post               460
MA Email           171
Facebook            33
Fax                 11
Social Media         8
Twitter              6
SRM Engage           1
Name: CHANNEL, dtype: int64
United States       68235
India               28746
Italy               16578
Philippines         15557
United Kingdom      12767
                    ...  
Greenland               1
Mauritania              1
South Sudan             1
Christmas Island        1
Vanuatu                 1
Name: CONTACT_COUNTRY, Length: 182, dtype: int64
Author Profile Correction    99485
Review                       96511
Ordering                     39062
Submission                   35078
Access                       27405
                             ...  
Resources                        1
Oracle Service Cloud             1
Journals Process Map             1
Category 2             

Techmail - Service Mailbox     314192
Connect Web Services - SOAP     93007
End-User Connect                73041
Incident Editor                 53639
Live Chat                       31305
Contact Editor                   5240
Ask A Question                   3885
Survey Flow Action                464
Feedback on Answers               178
Connect PHP                        53
Web Console Incident Editor         1
Name: SOURCE_LVL2_DESC, dtype: int64
Solved        538050
Unresolved     35522
Waiting         1434
Name: STATUS, dtype: int64


### Summary of the dataframe

In [5]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
23,RESOLUTION_CODE_LVL3_DESC,"[[nan, Parity, Manual, Author name, [Please de...",6,object,100.00
22,RESOLUTION_CODE_LVL2_DESC,"[[nan, Missing manuscript/article elements, Co...",29,object,99.54
4,CONTACT_REASON_LVL3_DESC,"[[nan, Force close CPC resupply, Force close r...",116,object,99.49
7,CREATED_TO_INITIAL_RESPONSE_DAYS,"[[nan, 0.8340000000000001, 1.93, 21.924, 0.059...",12143,float64,75.18
5,CONTACT_TYPE,"[[nan, Individual (B2C), Employee of Supplier,...",14,object,72.06
17,INCIDENT_SYSTEM,"[[nan, Evise, EES, PTS, Not Applicable, EM, ID...",27,object,62.40
1,CONTACT_COUNTRY,"[[nan, United States, Greece, Italy, India, Po...",183,object,56.69
3,CONTACT_REASON_LVL2_DESC,"[[Error message, Setup / manage, Additional in...",278,object,14.19
2,CONTACT_REASON_LVL1_DESC,"[[Using the product, Access, Review, nan, Auth...",84,object,12.90
10,CUSTOMER_CLASSIFICATION_TYPE,"[[Educator/faculty, Editor, Other (internal), ...",45,object,12.05


# Identify Missing Data

Missing values affect the performance of the machine learning models and its useful to identify and drop or impute missing values before modelling.

We will quantify the missing data and drop any columns lower than the given threshold. we have set threshold of 70% so any columns and rows with missing data over 70% are dropped.

In [6]:
#set threshold
threshold = 0.7


#Dropping columns with missing value rate higher than threshold
temp = df[df.columns[df.isnull().mean() < threshold]]

print('-------------------- Dropping Columns with missing data --------------------')
print(f'Following {len(df.columns.difference(temp.columns))} columns have missing data over the threshold and will be removed')
print(df.columns.difference(temp.columns))

df = temp

print('-------------------- Dropping rows with missing data --------------------')
print(f' There are {sum(df.isnull().mean(axis=1) > threshold)} rows with missing data over the threshold')

#Dropping rows with missing value rate higher than threshold
df = df.loc[df.isnull().mean(axis=1) < threshold]

# remove temp object from memory
del temp

#manually remove borderline case
df = df.drop(columns=[])

-------------------- Dropping Columns with missing data --------------------
Following 5 columns have missing data over the threshold and will be removed
Index(['CONTACT_REASON_LVL3_DESC', 'CONTACT_TYPE',
       'CREATED_TO_INITIAL_RESPONSE_DAYS', 'RESOLUTION_CODE_LVL2_DESC',
       'RESOLUTION_CODE_LVL3_DESC'],
      dtype='object')
-------------------- Dropping rows with missing data --------------------
 There are 0 rows with missing data over the threshold


In [7]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(get_data_frame_summmary(df))

,index,unique_values,unique_counts,data_type,percent_missing
14,INCIDENT_SYSTEM,"[[nan, Evise, EES, PTS, Not Applicable, EM, ID...",27,object,62.40
1,CONTACT_COUNTRY,"[[nan, United States, Greece, Italy, India, Po...",183,object,56.69
3,CONTACT_REASON_LVL2_DESC,"[[Error message, Setup / manage, Additional in...",278,object,14.19
2,CONTACT_REASON_LVL1_DESC,"[[Using the product, Access, Review, nan, Auth...",84,object,12.90
7,CUSTOMER_CLASSIFICATION_TYPE,"[[Educator/faculty, Editor, Other (internal), ...",45,object,12.05
16,OWNER_ID,"[[HOCS Education REPH 1LS, GJP Journal Manager...",109,object,11.98
17,OWNER_NAME,"[[Switzel Marie Jaena, Latha Sudarsanam, nan, ...",1580,object,11.98
18,RESOLUTION_CODE_LVL1_DESC,"[[Provided product information/doc, Didn't use...",344,object,11.06
5,CUSTOMER_CLASSIFICATION_PRODUCT,"[[HESI - iNet Testing, HESI Legacy, Journal, N...",187,object,8.40
4,CREATED_TO_CLOSED_DAYS,"[[0.0, 0.8340000000000001, 0.589, 1.374, 6.036...",31597,float64,6.43


### Data Imputation

We can impute missing data with meaningful data so that model development has good quality of data

We can impute categorical variables with the most frequestly occuring value and impute numerical variables with 0 or mean or median depending on the variable context.

INCIDENT_SYSTEM - Impute with most frequent

CONTACT_COUNTRY - Impute with most frequent

CONTACT_REASON_LVL2_DESC - Impute with most frequent

CONTACT_REASON_LVL1_DESC - Impute with most frequent

CUSTOMER_CLASSIFICATION_TYPE - Impute with most frequent

CUSTOMER_CLASSIFICATION_PRODUCT - Impute with most frequent

CREATED_TO_CLOSED_DAYS - Impute with median 

CHANNEL - Impute with most frequent

CUSTOMER_CLASSIFICATION_ROLE - Impute with most frequent


In [8]:
# #Filling all missing values with 0
# # data = data.fillna(0)

# #Filling missing values with the most frequest values
# df['INCIDENT_SYSTEM'].fillna(
#     df['INCIDENT_SYSTEM'].value_counts().idxmax(), inplace=True)

# df['CONTACT_COUNTRY'].fillna(
#     df['CONTACT_COUNTRY'].value_counts().idxmax(), inplace=True)

# df['CONTACT_REASON_LVL2_DESC'].fillna(
#     df['CONTACT_REASON_LVL2_DESC'].value_counts().idxmax(), inplace=True)

# df['CONTACT_REASON_LVL1_DESC'].fillna(
#     df['CONTACT_REASON_LVL1_DESC'].value_counts().idxmax(), inplace=True)

# df['CUSTOMER_CLASSIFICATION_TYPE'].fillna(
#     df['CUSTOMER_CLASSIFICATION_TYPE'].value_counts().idxmax(), inplace=True)

# df['CUSTOMER_CLASSIFICATION_PRODUCT'].fillna(
#     df['CUSTOMER_CLASSIFICATION_PRODUCT'].value_counts().idxmax(), inplace=True)

# df['CREATED_TO_CLOSED_DAYS'].fillna(
#     df['CREATED_TO_CLOSED_DAYS'].median(), inplace=True)

# df['CHANNEL'].fillna(
#     df['CHANNEL'].value_counts().idxmax(), inplace=True)

# df['CUSTOMER_CLASSIFICATION_ROLE'].fillna(
#     df['CUSTOMER_CLASSIFICATION_ROLE'].value_counts().idxmax(), inplace=True)

# Handling Outliers

Extreme value can skew the data distribution and thus affect the model development we identify outliers in numeric variables and handle them by removing or capping.

### Outlier Detection with Percentiles

In [9]:
#Dropping the outlier rows with Percentiles

df = drop_outliers(df, exclude = ['INCIDENT_ID', 'NUMBER_OF_RESPONSES'])

Dropping outliers for CREATED_TO_CLOSED_DAYS upper limit = 14.450799999999987 and lower limit = 0.0


# Binning

The main motivation of binning is to make the model more robust and prevent overfitting, however, it has a cost to the performance. Every time you bin something, you sacrifice information and make your data more regularized

The trade-off between performance and overfitting is the key point of the binning process


For Categorical variables the labels with low frequencies probably affect the robustness of statistical models negatively. Thus, assigning a general category to these less frequent values helps to keep the robustness of the model.

it is a good option to unite the labels with a count less than 100 to a new category like “Other”.

In [10]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)   

### Columns with less than 100 unique categories

In [11]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])
    

,index,unique_values,unique_counts,data_type,percent_missing
21,STATUS,[[Solved]],1,object,0.00
13,INCIDENT_REOPENED,"[[No, Yes]]",2,object,0.00
9,INCIDENT_AUTO_SOLVED,"[[No, Yes]]",2,object,0.00
19,SOURCE_LVL1_DESC,"[[Utilities, Public API, End-User pages, CX Co...",6,object,0.00
20,SOURCE_LVL2_DESC,"[[Techmail - Service Mailbox, Connect Web Serv...",11,object,0.00
0,CHANNEL,"[[CSS Email, nan, Web, Chat, Phone, Email, Cal...",14,object,4.31
6,CUSTOMER_CLASSIFICATION_ROLE,"[[Researcher, Content, Print, Organization, na...",15,object,2.78
15,NUMBER_OF_RESPONSES,"[[1, 0, 3, 2, 6, 4, 5, 7, 9, 8, 13, 11, 10, 14...",16,int64,0.00
14,INCIDENT_SYSTEM,"[[Evise, nan, EES, PTS, Not Applicable, EM, We...",27,object,54.96
7,CUSTOMER_CLASSIFICATION_TYPE,"[[Editor, Other (internal), Other (customer), ...",42,object,6.41


In [12]:
columns_to_bin = binning_summary[binning_summary['unique_counts'] < 100]['index'].to_list()

In [13]:
# select columns with less than 100 labels
df = df.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name in columns_to_bin else x)

### After Binning - Columns with less than 100 unique categories

In [14]:
binning_summary = get_data_frame_summmary(df, 
                                    sort_by='unique_counts', 
                                    ascending=True)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(binning_summary[binning_summary['unique_counts'] < 100])

,index,unique_values,unique_counts,data_type,percent_missing
21,STATUS,[[Solved]],1,object,0.00
13,INCIDENT_REOPENED,"[[No, Yes]]",2,object,0.00
9,INCIDENT_AUTO_SOLVED,"[[No, Yes]]",2,object,0.00
19,SOURCE_LVL1_DESC,"[[Utilities, Public API, End-User pages, CX Co...",6,object,0.00
20,SOURCE_LVL2_DESC,"[[Techmail - Service Mailbox, Connect Web Serv...",11,object,0.00
15,NUMBER_OF_RESPONSES,"[[1, 0, 3, 2, 6, 4, 5, 7, 9, 8, Other, 11]]",12,object,0.00
0,CHANNEL,"[[CSS Email, nan, Web, Chat, Phone, Email, Cal...",12,object,4.31
6,CUSTOMER_CLASSIFICATION_ROLE,"[[Researcher, Content, Print, Organization, na...",13,object,2.78
14,INCIDENT_SYSTEM,"[[Evise, nan, EES, PTS, Not Applicable, EM, We...",25,object,54.96
7,CUSTOMER_CLASSIFICATION_TYPE,"[[Editor, Other (internal), Other (customer), ...",30,object,6.41


## Log Transform

Logarithm transformation (or log transform) is one of the most commonly used mathematical transformations

It helps to handle skewed data and after transformation, the distribution becomes more approximate to normal.
In most of the cases the magnitude order of the data changes within the range of the data. log transform normalizes the magnitude differences.

It also decreases the effect of the outliers, due to the normalization of magnitude differences and the model become more robust.

Important note: The data you apply log transform must have only positive values, otherwise you receive an error. Also, you can add 1 to your data before transform it. Thus, you ensure the output of the transformation to be positive.


In [15]:
# No Log transform applied 

## Scaling

In most cases, the numerical features of the dataset do not have a certain range and they differ from each other. Scaling solves this problem. The continuous features become identical in terms of the range, after a scaling process. This process is not mandatory for many algorithms, but it might be still nice to apply. 

### Normalization

Normalization (or min-max normalization) scale all values in a fixed range between 0 and 1. This transformation does not change the distribution of the feature and due to the decreased standard deviations, the effects of the outliers increases. Therefore, before normalization, it is recommended to handle the outliers

### Standardization
Standardization (or z-score normalization) scales the values while taking into account standard deviation. If the standard deviation of features is different, their range also would differ from each other. This reduces the effect of the outliers in the features.
In the following formula of standardization, the mean is shown as μ and the standard deviation is shown as σ.

In [16]:
df = scale_numeric_features(df, inplace=True)

********************* - Scaling following 2 features - **********************
 CREATED_TO_CLOSED_DAYS standardized to CREATED_TO_CLOSED_DAYS
 INCIDENT_ID standardized to INCIDENT_ID


## One-hot encoding

This method spreads the values in a column to multiple flag columns and assigns 0 or 1 to them. These binary values express the relationship between grouped and encoded column.

This method changes your categorical data, which is challenging to understand for algorithms, to a numerical format

If you have N distinct values in the column, it is enough to map them to N-1, as the missing value can be deduced from the other columns



In [17]:
# print(columns_to_bin)

In [18]:
# # Specify Columns to encode
# columns_to_exclude = []
# columns_to_include = columns_to_bin
# columns_to_encode = remove_items(columns_to_include, columns_to_exclude)
# columns_to_encode

In [19]:
# for column in columns_to_encode:
#     encoded_columns = pd.get_dummies(df[column])
#     print(f'Encoding collumns : {column} to {len(encoded_columns.columns)} new encoded columns')
#     df = df.join(encoded_columns, rsuffix='_'+column).drop(column, axis=1)

In [20]:
# df.shape

In [21]:
df.head()

,CHANNEL,CONTACT_COUNTRY,CONTACT_REASON_LVL1_DESC,CONTACT_REASON_LVL2_DESC,CREATED_TO_CLOSED_DAYS,CUSTOMER_CLASSIFICATION_PRODUCT,CUSTOMER_CLASSIFICATION_ROLE,CUSTOMER_CLASSIFICATION_TYPE,ECR_ID,INCIDENT_AUTO_SOLVED,...,INCIDENT_ID,INCIDENT_REOPENED,INCIDENT_SYSTEM,NUMBER_OF_RESPONSES,OWNER_ID,OWNER_NAME,RESOLUTION_CODE_LVL1_DESC,SOURCE_LVL1_DESC,SOURCE_LVL2_DESC,STATUS
2,CSS Email,United States,Review,Additional info required,-0.57,Journal,Researcher,Editor,ECR-938,No,...,-0.57,No,Evise,1,GJP Journal Managers Chennai,Latha Sudarsanam,Didn't use KB article,Utilities,Techmail - Service Mailbox,Solved
3,CSS Email,NaN,NaN,NaN,-0.66,No Product,Content,Other (internal),ECR-10018297,No,...,0.67,No,NaN,0,NaN,NaN,NaN,Utilities,Techmail - Service Mailbox,Solved
4,NaN,NaN,Author Profile Correction,Reprofile,-0.39,Scopus,Content,Other (customer),ECR-10052132,No,...,-0.24,No,NaN,0,NaN,NaN,AFW tool,Public API,Connect Web Services - SOAP,Solved
5,Web,NaN,Add Missing Document,Missing Issues,1.15,Scopus,Content,Other (customer),ECR-442754,No,...,-0.91,No,NaN,0,SCOPUS Gen CHN 1LS,Shanthini Sankar,Sent to Supplier-Missing article,End-User pages,End-User Connect,Solved
6,CSS Email,NaN,NaN,NaN,-0.03,No Product,Content,Other (internal),ECR-10018297,No,...,0.68,No,NaN,0,NaN,NaN,NaN,Utilities,Techmail - Service Mailbox,Solved


## Transform Date columns

Recency and Frequency Variables and length / duration.


In [22]:
# convert date columns to date data type

date_columns = ['INCIDENT_CREATED_DATETIME', 'INCIDENT_CLOSED_DATETIME']

In [23]:
for column in date_columns:
    df[column] = pd.to_datetime(df[column])

In [24]:
# calcualte new variable subscription_length and agreement_length
df['incident_length_to_close'] = months_between_columns(df['INCIDENT_CLOSED_DATETIME'],df['INCIDENT_CREATED_DATETIME'])

In [25]:
 df.head()

,CHANNEL,CONTACT_COUNTRY,CONTACT_REASON_LVL1_DESC,CONTACT_REASON_LVL2_DESC,CREATED_TO_CLOSED_DAYS,CUSTOMER_CLASSIFICATION_PRODUCT,CUSTOMER_CLASSIFICATION_ROLE,CUSTOMER_CLASSIFICATION_TYPE,ECR_ID,INCIDENT_AUTO_SOLVED,...,INCIDENT_REOPENED,INCIDENT_SYSTEM,NUMBER_OF_RESPONSES,OWNER_ID,OWNER_NAME,RESOLUTION_CODE_LVL1_DESC,SOURCE_LVL1_DESC,SOURCE_LVL2_DESC,STATUS,incident_length_to_close
2,CSS Email,United States,Review,Additional info required,-0.57,Journal,Researcher,Editor,ECR-938,No,...,No,Evise,1,GJP Journal Managers Chennai,Latha Sudarsanam,Didn't use KB article,Utilities,Techmail - Service Mailbox,Solved,0.00
3,CSS Email,NaN,NaN,NaN,-0.66,No Product,Content,Other (internal),ECR-10018297,No,...,No,NaN,0,NaN,NaN,NaN,Utilities,Techmail - Service Mailbox,Solved,0.00
4,NaN,NaN,Author Profile Correction,Reprofile,-0.39,Scopus,Content,Other (customer),ECR-10052132,No,...,No,NaN,0,NaN,NaN,AFW tool,Public API,Connect Web Services - SOAP,Solved,0.00
5,Web,NaN,Add Missing Document,Missing Issues,1.15,Scopus,Content,Other (customer),ECR-442754,No,...,No,NaN,0,SCOPUS Gen CHN 1LS,Shanthini Sankar,Sent to Supplier-Missing article,End-User pages,End-User Connect,Solved,0.00
6,CSS Email,NaN,NaN,NaN,-0.03,No Product,Content,Other (internal),ECR-10018297,No,...,No,NaN,0,NaN,NaN,NaN,Utilities,Techmail - Service Mailbox,Solved,0.00


## Remove unused and/or redundant Features

Unused features are those that don’t make sense to pass into our machine learning algorithms. such as ID columns
Features that wouldn't be available at the time of prediction, Other text descriptions

Redundant features would typically be those that have been replaced by other features that you’ve added during feature engineering


In [26]:
columns_to_remove = ['CONTACT_REASON_LVL2_DESC', 'OWNER_NAME', 'RESOLUTION_CODE_LVL1_DESC', 
                     'CUSTOMER_CLASSIFICATION_PRODUCT', 'INCIDENT_CLOSED_DATETIME', 'INCIDENT_CREATED_DATETIME']

In [27]:
df = df.drop(columns_to_remove, axis=1)

In [28]:
df.head()

,CHANNEL,CONTACT_COUNTRY,CONTACT_REASON_LVL1_DESC,CREATED_TO_CLOSED_DAYS,CUSTOMER_CLASSIFICATION_ROLE,CUSTOMER_CLASSIFICATION_TYPE,ECR_ID,INCIDENT_AUTO_SOLVED,INCIDENT_ID,INCIDENT_REOPENED,INCIDENT_SYSTEM,NUMBER_OF_RESPONSES,OWNER_ID,SOURCE_LVL1_DESC,SOURCE_LVL2_DESC,STATUS,incident_length_to_close
2,CSS Email,United States,Review,-0.57,Researcher,Editor,ECR-938,No,-0.57,No,Evise,1,GJP Journal Managers Chennai,Utilities,Techmail - Service Mailbox,Solved,0.00
3,CSS Email,NaN,NaN,-0.66,Content,Other (internal),ECR-10018297,No,0.67,No,NaN,0,NaN,Utilities,Techmail - Service Mailbox,Solved,0.00
4,NaN,NaN,Author Profile Correction,-0.39,Content,Other (customer),ECR-10052132,No,-0.24,No,NaN,0,NaN,Public API,Connect Web Services - SOAP,Solved,0.00
5,Web,NaN,Add Missing Document,1.15,Content,Other (customer),ECR-442754,No,-0.91,No,NaN,0,SCOPUS Gen CHN 1LS,End-User pages,End-User Connect,Solved,0.00
6,CSS Email,NaN,NaN,-0.03,Content,Other (internal),ECR-10018297,No,0.68,No,NaN,0,NaN,Utilities,Techmail - Service Mailbox,Solved,0.00


Save clean pickle files

In [29]:
df.to_pickle("../"+cfg['clean_pickles_for_graphs_dir']+"/"+cfg['interactions_file']+"_for_graphs.pickle")